In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote as rcq
import calculate_trends as ct

In [2]:
roll_list = [20]
mom_list  = [20]
rsi_list  = [15,30]

In [3]:
tempDf = rcq.read_quote_data('IBM')

rcq.fill_quote_gaps( tempDf )
rcq.fix_splits( tempDf )

In [8]:
diffs = ct.generate_differentials(   tempDf                   ) # Differentials between c,ac, o, h, l, v
rolls = ct.generate_rolling_close(   tempDf,        roll_list ) # Rolling averages of adj c, listed as C_mean_n and C_std_10
moms  = ct.generate_momentum_close(  tempDf,         mom_list ) # Momentum between day and days offset in list, Momentum_n
rsis  = ct.generate_rsi(             tempDf,         rsi_list ) # Relative Strength Indexes over n days, RSI_n
bolls = ct.generate_bollinger_bands( tempDf, rolls, roll_list ) # percentB of Bollinger bands, 1 is at upper, 0 at lower

In [6]:
# Diff_co goes to +/- .05
# Diff_v goes to -1 +2
# Momentum_5 +/- .15
# Momentum_15 -.25 +.25
# Momentum_30 -.3 +.25
# RSI naturally normalized to 0-100

In [15]:
reload( ct )

<module 'calculate_trends' from 'calculate_trends.py'>

In [16]:
#norm_diffs = diffs.copy()
#norm_moms  =  moms.copy()
norm_rsis  =  rsis.copy()

#norm_diffs['Diff_co'] = ct.normalize_column( norm_diffs, 'Diff_co', minVal=-0.05, maxVal=0.05 )
#norm_diffs['Diff_v' ] = ct.normalize_column( norm_diffs, 'Diff_v' , minVal=-1.00, maxVal=2.00 )

#norm_moms['Momentum_20'] = ct.normalize_column( norm_moms, 'Momentum_20', minVal=-0.15, maxVal=0.15 )

norm_rsis['RSI_15'] = ct.normalize_column( norm_rsis, 'RSI_15', minVal=0, maxVal=100 )
norm_rsis['RSI_30'] = ct.normalize_column( norm_rsis, 'RSI_30', minVal=0, maxVal=100 )

               RSI_15     RSI_30
Date                            
2017-03-01  51.386561  56.364837
2017-02-28  52.598069  54.784501
2017-02-27  51.623306  56.544208
2017-02-24  57.340423  57.628294
2017-02-23  52.038962  56.982568
              RSI_15     RSI_30
Date                           
2017-03-01  0.513866  56.364837
2017-02-28  0.525981  54.784501
2017-02-27  0.516233  56.544208
2017-02-24  0.573404  57.628294
2017-02-23  0.520390  56.982568
 
              RSI_15     RSI_30
Date                           
2017-03-01  0.513866  56.364837
2017-02-28  0.525981  54.784501
2017-02-27  0.516233  56.544208
2017-02-24  0.573404  57.628294
2017-02-23  0.520390  56.982568
              RSI_15    RSI_30
Date                          
2017-03-01  0.513866  0.563648
2017-02-28  0.525981  0.547845
2017-02-27  0.516233  0.565442
2017-02-24  0.573404  0.576283
2017-02-23  0.520390  0.569826
 


In [17]:
norm_rsis.head()

,RSI_15,RSI_30
Date,,
2017-03-01,0.513866,0.563648
2017-02-28,0.525981,0.547845
2017-02-27,0.516233,0.565442
2017-02-24,0.573404,0.576283
2017-02-23,0.520390,0.569826


In [7]:
# Join everything, or can leave as seperate 
tempDf = tempDf.join( norm_diffs , how='inner' )
tempDf = tempDf.join( rolls      , how='inner' )
tempDf = tempDf.join( norm_moms  , how='inner' )
tempDf = tempDf.join( norm_rsis  , how='inner' )
tempDf = tempDf.join( bolls      , how='inner' )
tempDf = tempDf.drop( ['Open','High','Low','Volume','Close'], axis=1 )

In [8]:
tempDf.head()

,Adj Close,Diff_co,Diff_v,Close_mean_20,Close_std_20,Momentum_20,RSI_15,RSI_30,Bollinger_20
Date,,,,,,,,,
2017-03-01,181.949997,0.581450,0.581450,178.651213,2.990200,0.669391,0.513866,0.513866,0.775800
2017-02-28,179.820007,0.524529,0.524529,178.211260,3.123634,0.603182,0.525981,0.525981,0.628756
2017-02-27,179.399994,0.401208,0.401208,177.941303,3.209412,0.566149,0.516233,0.516233,0.613626
2017-02-24,181.350006,0.563259,0.563259,177.766759,3.220811,0.576942,0.573404,0.573404,0.778132
2017-02-23,181.649994,0.480769,0.480769,177.562180,3.109244,0.589672,0.520390,0.520390,0.828682


In [16]:
tempDf.tail(31)

,Adj Close,Diff_co,Diff_v,Close_mean_20,Close_std_20,Momentum_20,RSI_15,RSI_30,Bollinger_20
Date,,,,,,,,,
2004-02-17,77.166450,0.437994,0.437994,76.711591,0.771346,0.647261,0.310345,0.310345,0.647424
2004-02-13,77.430477,0.561554,0.561554,76.548348,0.985707,0.707461,0.338279,0.338279,0.723730
2004-02-12,77.112092,0.424046,0.424046,76.321509,1.256770,0.837762,0.337564,0.337564,0.657265
2004-02-11,77.624616,0.576613,0.576613,75.966771,1.872385,0.887293,0.362897,0.362897,0.721355
2004-02-10,77.352822,0.617827,0.617827,75.562761,2.315219,0.799344,0.372013,0.372013,0.693293
2004-02-09,76.840292,0.463750,0.463750,75.244055,2.488101,0.788726,0.409842,0.409842,0.660387
2004-02-06,76.832531,0.509105,0.509105,74.937796,2.652802,0.717125,0.448133,0.448133,0.678560
2004-02-05,76.646155,0.386000,0.386000,74.702865,2.684043,0.718438,0.533333,0.533333,0.681004
2004-02-04,77.677306,0.581506,0.581506,74.467174,2.711245,0.755391,0.552753,0.552753,0.796002


In [8]:
import matplotlib.pyplot as plt

In [15]:
plt.figure()
tempDf.plot(kind='scatter',x='RSI_15',y='RSI_30')
plt.show()

/home/sean/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [10]:
from sklearn.decomposition import PCA

In [14]:
my_pca = PCA( n_components=4 )
my_pca.fit( tempDf.drop(['Adj Close','Close_mean_20','Close_std_20','Diff_co','Diff_v'],axis=1) )
print my_pca.explained_variance_ratio_

[  8.23914226e-01   1.07413384e-01   6.86723901e-02   4.10898555e-33]


In [8]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [8]:
print tempDf.shape[0]

3313


In [33]:
max_day = max( [max(roll_list),max(mom_list),max(rsi_list)] )
N_train = 2500
N_test  = 700
off     = 5

# Put things in right cronological order
flipped_df = tempDf.copy()
flipped_df = flipped_df[::-1]
flipped_df = flipped_df[max_day:]

# Labels need to be stored as a list, because will use diff CLF for each
train_label_list   = []
test_label_list    = []

# Features are constant for each clf
train_features = flipped_df[:N_train]
test_features  = flipped_df[ N_train+off:N_train+off+N_test]

# Loop over offsets so have training data for each day
for offset in range( 1, off+1 ):
    
    # Train data from 0 to N_train
    # Train labels from ^+offset
    # Test data from N_train+o to N_train + N_test + o
    # Test labels from ^+offset
    
    train_label_list.append(  flipped_df.ix[offset:N_train+offset,'Adj Close'].values  )
    test_label_list.append(   flipped_df.ix[2*offset+N_train::N_test+N_train+2*offset,'Adj Close'].values )

In [30]:
test_features['Adj Close'].head()

Date
2014-01-29    159.390530
2014-01-30    160.257968
2014-01-31    159.643530
2014-02-03    156.228020
2014-02-04    156.173808
Name: Adj Close, dtype: float64

In [32]:
clf_list = []

for i in range( 0, off ):
    
    clf_list.append( SVR() )
    
    clf_list[i].fit( train_features, train_label_list[i] )
    
    print clf_list[i].score( train_features, train_label_list[i] ), test_features.ix[0,'Adj Close'],
    print clf_list[i].predict( test_features.iloc[0] ), test_label_list[i][0]

0.925266725968 159.39053 [ 154.86204236] 162.318117


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.924107025419 159.39053 [ 154.93927547] 159.79715


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.922888925438 159.39053 [ 154.56218893] 160.257968


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.921903704721 159.39053 [ 154.84077323] 156.22802


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.91930927933 159.39053 [ 153.77003668] 157.43882


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
